In [2]:
from os import listdir
from os.path import isfile
from fnmatch import fnmatch
from math import isnan, ceil, factorial
from pprint import pprint
from statistics import median
from re import match, search

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.font_manager import FontProperties

In [3]:
def dump_csv(df, filename):
    df.to_csv(filename, float_format='%.4f')
    return

In [4]:
def p_0(rho, m):
    frac = ((m * rho) ** m) / (factorial(m) * (1.0 - rho))
    print(frac)
    rng = [float(n) for n in range(1, int(m))]
    sum_vals = [((m * rho) ** n) / factorial(n) for n in rng]
    return 1.0 / (1.0 + frac + sum(sum_vals))

In [37]:
ratios = ['0:1']
runs = [x for x in range(1, 3 + 1)] 
instances = [x for x in range(1, 2 + 1)]
threads = 1 
clients = [x for x in range(1, 3 + 1)]
vclients = [1, 4, 8, 16, 24, 32, 64]
worker_threads = [32]
mws = [x for x in range(1, 1 + 1)]

vclients = [4, 64]
mw_multi_index = [ratios, worker_threads, vclients]
mw_len_index = len(ratios) * len(worker_threads) * len(vclients)

mw_names=['ratio', 'WTS', 'VCs']

mw_index = pd.MultiIndex.from_product(mw_multi_index, names=mw_names)
columns = ['Throughput (ops/s)', 'ResponseTime (ms)']

columns += ['Jobs', 'ServiceTime (ms)', 'WaitingTime (ms)', 'QueueSize']

df = pd.read_csv('../baseline_mw_1_mw/out/middleware_data.csv', index_col=[0,1,2,3,4])
df1 = pd.DataFrame([], index=mw_index, columns=columns)

for ratio in ratios:
    for wt in worker_threads:
        for vc in vclients:
            row = []
            for c in columns:
                if 'ops/s' in c or 'Jobs' in c or 'Queue' in c:
                    row.append(df.unstack().loc[(ratio, wt, vc), c].T.sum().mean())
                else:
                    row.append(df.unstack().loc[(ratio, wt, vc), c].T.mean().mean())
            df1.loc[(ratio, wt, vc)] = row
            
#Service rate is considered to be the max TP for each WT config, since, it constitutes a reasonable lower bound. 


# df1['V_Server'] = pd.Series()
# df1['TP Server'] = pd.Series()
# df1['Rho_Server'] = pd.Series()
# df1['V_Worker'] = pd.Series()
# df1['TP_Worker'] = pd.Series()
# df1['Rho_Server'] = pd.Series()
# df1['V_Parser'] = pd.Series()
# df1['TP_Parser'] = pd.Series()
# df1['Rho_Parser'] = pd.Series()

service = False

for ratio in ratios:
    for wt in worker_threads:
        for vc in vclients:
            if service:
                df1.loc[(ratio, wt, vc), 'ServiceRate (ops/s)'] = df.unstack().loc[(ratio, wt, vc), 'Throughput (ops/s)'].T.sum().max()
            else:
                df1.loc[(ratio, wt, vc), 'ServiceRate (ops/s)'] = 1 / (df.unstack().loc[(ratio, wt, vc), 'ServiceTime (ms)'].T.mean().mean() / 1000)
                
            throughput = df.unstack().loc[(ratio, wt, vc), 'Throughput (ops/s)'].T.sum().max()
            service_time = df.unstack().loc[(ratio, wt, vc), 'ServiceTime (ms)'].T.mean().mean() / 1000
            df1.loc[(ratio, wt, vc), 'V_Server'] = 1
            df1.loc[(ratio, wt, vc), 'TP_Server'] = df1.loc[(ratio, wt, vc), 'V_Server'] * throughput 
#             df1.loc[(ratio, wt, vc), 'Rho_Server'] = df1.loc[(ratio, wt, vc), 'TP_Server'] * service_time
            
            df1.loc[(ratio, wt, vc), 'V_Worker'] = 1 / wt
            df1.loc[(ratio, wt, vc), 'TP_Worker'] = df1.loc[(ratio, wt, vc), 'V_Worker'] * throughput 
            df1.loc[(ratio, wt, vc), 'Rho_Worker'] = df1.loc[(ratio, wt, vc), 'TP_Worker'] * service_time
            
            df1.loc[(ratio, wt, vc), 'V_Parser'] = 1
            df1.loc[(ratio, wt, vc), 'TP_Parser'] = df1.loc[(ratio, wt, vc), 'V_Parser'] * throughput 
#             df1.loc[(ratio, wt, vc), 'Rho_Parser'] = df1.loc[(ratio, wt, vc), 'TP_Parser'] * service_time

 

# df1.reset_index(level=1, drop=True, inplace=True)
# df1['WTs'] = worker_threads
# Utilization
# Probability of queuing

# # Expected number of Jobs in the system (w + s)
# df1['En'] = df1['Rho'] / (1 - df1['Rho'])
# # Expected number of Jobs in the queue(w)
# df1['En_q'] = pow(df1['Rho'], 2) / (1 - df1['Rho'])
# # Expected AVG Waiting Time in the queue(w)
# df1['Ew'] = df1['Rho'] * (1 / df1['ServiceRate (ops/s)']) / (1.0 - df1['Rho']) * 1000
# # Expected AVG Response Time
# df1['Er'] = (1.0 / (df1['ServiceRate (ops/s)'] * (1.0 - df1['Rho']))) * 1000

# dump_csv(df1, 'out/mm1.csv')
df1 = df1.drop(['ResponseTime (ms)', 'Jobs', 'WaitingTime (ms)', 'QueueSize', 'ServiceRate (ops/s)'], axis =1)


print(df1.T.to_latex())

\begin{tabular}{lll}
\toprule
ratio & \multicolumn{2}{l}{0:1} \\
WTS & \multicolumn{2}{l}{32} \\
VCs &        4  &        64 \\
\midrule
Throughput (ops/s) &   2930.21 &   2931.97 \\
ServiceTime (ms)   &   5.70667 &   10.8233 \\
V\_Server           &         1 &         1 \\
TP\_Server          &   2930.54 &   2932.92 \\
V\_Worker           &   0.03125 &   0.03125 \\
TP\_Worker          &   91.5794 &   91.6538 \\
Rho\_Worker         &  0.522613 &  0.991999 \\
V\_Parser           &         1 &         1 \\
TP\_Parser          &   2930.54 &   2932.92 \\
\bottomrule
\end{tabular}

